# ARCS Ei=100meV resolution

Instrument: ARCS
* Ei=100meV
* Fermi chopper: 600 Hz
* T0: 120Hz

Sample: resolution sample
* hkl: -16/3.,-8/3.,8/3.
* E: 40

In [2]:
# some goodies
# %matplotlib notebook
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

In [3]:
import histogram.hdf as hh, histogram as H

## Formula

\begin{align}
\frac{\Delta E_i}{E_i} & = 2 \frac{\Delta v_i}{v_i} \\
\frac{\Delta v_i}{v_i} & = \sqrt{ \big( \frac{\Delta t_i}{t_i} \big)^2 + \big(\frac{\Delta L_i}{L_i} \big)^2}
\end{align}

## Input Parameters

### Instrument parameters

In [39]:
Ei = 100
L_PM=11.61
R = 3.
L_PS=13.6
L_MS=L_PS-L_PM
sample_size = 10*.01 #

### Dynamics

In [5]:
E = 40.
hkl = -16/3.,-8/3.,8/3.

In [14]:
%%file Si.yml
name: Si
chemical_formula: Si2
lattice: 
 constants: 5.490700041, 5.490700041, 5.490700041, 90, 90, 90
 basis_vectors:
  - 5.490700041, 0, 0
  - 0, 5.490700041, 0
  - 0, 0, 5.490700041
 primitive_basis_vectors:
  - 0.0, 2.71526503565, 2.71526503565
  - 2.71526503565, 0.0, 2.71526503565
  - 2.71526503565, 2.71526503565, 0.0
excitations:
  - type: deltafunction
    hkl: -16/3.,-8/3.,8/3.
    E: 40.
    dE: 0.5
orientation:
 u: -1, 1, -1
 v: 2, 1, -1
shape: hollowCylinder in_radius="5./8*inch" out_radius="1.*inch" height="1.5*inch"
temperature: 100*K

Writing Si.yml


### Other parameters

In [44]:
m = 1.6750e-24 * 1e-3 #kg

### Derived parameters

In [6]:
from mcni.utils import conversion as Conv

In [7]:
vi = Conv.e2v(Ei)
print vi

4373.93313724


In [8]:
ti = L_PM/vi*1e6 # microsecond
print ti

2654.36156331


In [9]:
Ef = Ei - E

In [10]:
vf = Conv.e2v(Ef)
print vf

3388.03403959


In [13]:
!mcvine workflow sx orientation solve_psi --help

Usage: mcvine workflow sx orientation solve_psi [OPTIONS] SAMPLE

  compute psi angle

Options:
  --Ei FLOAT
  --hkl <FLOAT FLOAT FLOAT>...
  --E FLOAT
  --psimin FLOAT
  --psimax FLOAT
  --number-segments INTEGER
  --help                        Show this message and exit.


In [15]:
hkl_opt = '%s %s %s' % tuple(hkl)

In [24]:
!mcvine workflow sx orientation solve_psi --Ei=100 --hkl {hkl_opt} \
    --E 40 --psimin -5 --psimax 90 Si.yml

/home/lj7/dv/mcvine/export/share/mcvine/workflow/mcvine_workflow/singlextal/solve_psi.py:55: UserWarning: Traceback (most recent call last):
  File "/home/lj7/dv/mcvine/export/share/mcvine/workflow/mcvine_workflow/singlextal/solve_psi.py", line 52, in solve
    results.append(solver(res, min, max))
  File "/home/lj7/miniconda2/envs/dev-mcvine/lib/python2.7/site-packages/scipy/optimize/zeros.py", line 442, in brentq
    r = _zeros._brentq(f,a,b,xtol,rtol,maxiter,args,full_output,disp)
ValueError: f(a) and f(b) must have different signs

  warnings.warn(tb.format_exc())
psi=46.374678916, Q=[ 5.41072298 -5.15712343  0.        ]


In [28]:
Q =  np.array([5.41072298, -5.15712343,  0. ])

In [120]:
Q_len = np.linalg.norm(Q); print Q_len

7.47474716887


In [35]:
ki = Conv.e2k(Ei); print ki

6.94692092177


In [36]:
kiv = np.array([ki, 0, 0])
kfv = kiv - Q

** Verify the momentum and energy transfers **

In [38]:
print Ei-Conv.k2e(np.linalg.norm(kfv))
print Ei-Ef

39.9999999739
40.0


** Compute detector pixel position **

In [41]:
z = kfv[2]/(kfv[0]**2+kfv[1]**2)**.5 * R
L_SD=(z**2+R**2)**.5
print z, L_SD

0.0 3.0


## Differentials

In [45]:
pE_pt = -m*(vi**3/L_PM + vf**3/L_SD * L_MS/L_PM)

In [46]:
print pE_pt

-1.5794390036e-17


In [47]:
# convert to eV/microsecond
eV = 1.60218e-19
meV = eV*1e-3
mus = 1.e-6
pE_pt /= meV/mus
print pE_pt

-0.098580621628


In [48]:
pE_ptMD = m*vf**3/L_SD
pE_ptMD /= meV/mus
print pE_ptMD

0.135526912828


In [49]:
pE_pLPM = m/L_PM * (vi**2 + vf**3/vi * L_MS/ L_SD)
pE_pLPM /= meV
print pE_pLPM

22.5382095553


In [50]:
pE_pLMS= -m/L_SD * (vf**3/vi)
pE_pLMS /= meV
print pE_pLMS

-30.9851359349


In [51]:
pE_pLSD = -m*vf*vf/L_SD
pE_pLSD /= meV
print pE_pLSD

-40.0016384853


In [53]:
# we don't need pE_pLSD, instead we need pE_pR and pE_pz. R and z are cylinder radius and z coordinate
pE_pR = pE_pLSD * (R/L_SD)
pE_pz = pE_pLSD * (z/L_SD)
print pE_pR, pE_pz

-40.0016384853 -0.0


## Estimate of standard deviations

In [65]:
tau_P = 10 # microsecond
tau_M = 8 # microsecond
tau_D = 0.1 # microsecond

## Calculations

In [54]:
pE_p_vec = [pE_pt, pE_ptMD, pE_pLPM, pE_pLMS, pE_pR, pE_pz]

pE_p_vec = np.array(pE_p_vec)

J_E = pE_p_vec/E

In [57]:
print J_E

[-0.00246452  0.00338817  0.56345524 -0.7746284  -1.00004096 -0.        ]


In [63]:
sigma_t = (tau_P**2+tau_M**2)**.5

In [66]:
sigma_tMD = (tau_M**2+tau_D**2)**.5

In [67]:
div = 0.01
sigma_LPM = L_PM * div*div

In [68]:
# mainly due to sample size
sigma_LMS = 0.025

In [82]:
# mainly due to det tube diameter
sigma_R = 0.025

In [83]:
# pixel size
sigma_z = 1./128

In [84]:
sigma2 = np.diag([sigma_t, sigma_tMD, sigma_LPM, sigma_LMS, sigma_R, sigma_z])

In [85]:
print sigma2

[[  1.28062485e+01   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   8.00062498e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   1.16100000e-03   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   2.50000000e-02
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    2.50000000e-02   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   7.81250000e-03]]


In [86]:
cov = np.dot(J_E, np.dot(sigma2, J_E))

In [87]:
sigma_E = E*np.sqrt(cov)
print sigma_E

8.05396810261


* ** Note: this may be more like FWHM than sigma_E because of the approx I made **
* ** FWHM is 2.355 sigma **

In [88]:
FWHM_E = sigma_E
sigma_E = FWHM_E/2.355
print sigma_E

3.41994399262


## Include Q

In [89]:
print ti

2654.36156331


In [91]:
tf = L_MS/vf*1e6; print tf

587.361276996


In [92]:
thetai = 0
phii = 0

In [93]:
print R

3.0


In [96]:
eeta = np.arctan2(Q[1], Q[0])
print eeta

-0.761405490065


In [100]:
hbar= 1.0545718e-34

In [104]:
AA = 1e-10

In [105]:
from numpy import sin, cos

In [106]:
pQx_pt = -m/hbar*(L_PM/ti/ti/mus/mus*cos(thetai)*cos(phii)+R/tf/tf/mus/mus*L_MS/L_PM*cos(eeta))

In [107]:
print pQx_pt

-4.33096008289e+13


In [108]:
pQx_pt/=1./AA/mus
print pQx_pt

-0.00433096008289


In [109]:
pQx_ptMD = m/hbar * R/tf/tf * cos(eeta) / mus/mus
pQx_ptMD /= 1./AA/mus
print pQx_ptMD

0.00999788372299


In [111]:
pQx_pLPM = m/hbar *(cos(thetai) * cos(phii)/ti + ti/tf/tf * R*L_MS/L_PM/L_PM * cos(eeta)) / mus
pQx_pLPM /= 1./AA
print pQx_pLPM

0.990175191756


In [112]:
pQx_pLMS = -m/hbar * R/tf/tf*ti/L_PM*cos(eeta) / mus
pQx_pLMS /= 1./AA
print pQx_pLMS

-2.28578796458


In [113]:
pQx_pR = -m/hbar/tf*cos(eeta) / mus
pQx_pR /= 1./AA
print pQx_pR

-1.9574565836


In [114]:
pQx_peeta = m/hbar * R/tf*sin(eeta) /mus
pQx_peeta /= 1./AA
print pQx_peeta

-5.59713290504


In [117]:
pQx_pthetai = -m/hbar*L_PM/ti*sin(thetai)*cos(phii)/mus
pQx_pthetai/=1./AA
print pQx_pthetai

-0.0


In [119]:
pQx_pphii = -m/hbar*L_PM/ti*cos(thetai)*sin(phii)/mus
pQx_pphii/=1./AA
print pQx_pphii

-0.0


In [121]:
pQx_p_vec = [pQx_pt, pQx_ptMD, pQx_pLPM, pQx_pLMS, pQx_pR, 0, pQx_peeta, pQx_pthetai, pQx_pphii]
pQx_p_vec = np.array(pQx_p_vec)
J_Qx = pQx_p_vec/Q_len

**Qy**